#### 2018/11/1<br>V1.0_0.6231<br>因为在计算ctr的时候是使用了data.groupby().agg()的，所以过拟合现象严重！

#### <font color=red>2018/11/2<br>V1.2_0.7128<br>1.建立在V1.0版本上；<br>2.将data.groupby.agg改成了traindf.grouby.agg，因为data中有一部分的label是-1，所以对成绩产生了非常大的噪声；<br>3.将点击与否的界限从0.5改成了0.4；<br>4.将原先使用lgb.train改成了LGBMClassifier.fit，但是对成绩本身的影响应该不是特别大；<font>  

#### <font color=green>2018/11/3<br>V1.3<br>1.建立在V1.2的基础上；<br>1.对于部分0值变量重新赋予一个新值-1，从而方便使用scaler；<br>2.使用LogisticRegressor进行训练；<font>

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import gc
gc.collect()

0

In [2]:
data=pd.read_csv('../data/data.csv')

In [3]:
data.shape

(2099998, 5)

In [4]:
data['query_len']=data['query'].apply(lambda x:len(x.replace('{','').replace('}','').split(',')))
# print(data[data['label']!=-1].groupby('query_len')['label'].mean())
# print('====================')
data['prefix_len']=data['prefix'].apply(lambda x:len(str(x)))
# print(data[data['label']!=-1].groupby('prefix_len')['label'].mean())
# print('====================')
data['title_len']=data['title'].apply(lambda x:len(str(x)))
# print(data[data['label']!=-1].groupby('title_len')['label'].mean())
# print('====================')

In [5]:
gc.collect()

21

In [6]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
lr_col=['prefix','title','tag','query']
for feature in lr_col:
    data[feature]=data[feature].astype(str)

data['prefix']=le.fit_transform(data['prefix'])
data['title']=le.fit_transform(data['title'])
data['query']=le.fit_transform(data['query'])
data['tag']=le.fit_transform(data['tag'])

In [7]:
traindf=data[data['label']!=-1]
testdf=data[data['label']==-1]
print(traindf.shape)
print(testdf.shape)

(2049998, 8)
(50000, 8)


In [8]:
col=['prefix','query','title','tag','query_len','title_len']
for item in col:
    temp=traindf.groupby(item,as_index=False)['label'].agg({item+'_click':'sum',item+'_count':'count'})
    temp[item+'_ctr']=(temp[item+'_click']/temp[item+'_count'])
    traindf = pd.merge(traindf, temp, on=item, how='left')
    testdf=pd.merge(testdf,temp,on=item,how='left')

In [9]:
for i in range(len(col)):
    for j in range(i+1,len(col)):
        item_g=[col[i],col[j]]
        temp=traindf.groupby(item_g,as_index=False)['label'].agg({'_'.join(item_g)+'_click':'sum','_'.join(item_g)+'_count':'count'})
        temp['_'.join(item_g)+'_ctr']=temp['_'.join(item_g)+'_click']/(temp['_'.join(item_g)+'_count']+3)
        traindf=pd.merge(traindf,temp,on=item_g,how='left')
        testdf=pd.merge(testdf,temp,on=item_g,how='left')

In [10]:
print(traindf.shape,testdf.shape)

(2049998, 71) (50000, 71)


In [11]:
for fea in testdf.columns:
    testdf.loc[testdf[fea].isnull(),fea]=-1

# ===========Train===============

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import BaggingRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

testddf=testdf.drop(['label'],axis=1)

y=traindf.pop('label').values

X=traindf.as_matrix()
testX=testddf.as_matrix()

mm=MinMaxScaler()
X=mm.fit_transform(X)
testX=mm.transform(testX)

In [13]:
sub_train=pd.DataFrame()
sub_test=pd.DataFrame()
model=LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=0.08, random_state=1,
                            solver='liblinear', max_iter=100,verbose=1)

# model=BaggingRegressor(clf,n_estimators=2000, max_samples=0.8, 
#                        max_features=1.0, bootstrap=True, 
#                        bootstrap_features=False, n_jobs=-1)

In [14]:
N=10
skf=StratifiedKFold(n_splits=N,random_state=1,shuffle=True)
ft=0
fv=0
for i,(train_index,test_index) in enumerate(skf.split(X,y)):
    print("Fold=%s"%i)
    model.fit(X[train_index],y[train_index])
    train_pred=model.predict(X[train_index])
    trainf=f1_score(y[train_index],train_pred)
    print("f1 in train set:",trainf)
    ft+=trainf
    
    val_pred=model.predict(X[test_index])
    valf=f1_score(y[test_index],val_pred)
    print("f1 in test set:",valf)
    fv+=valf
    
    train_all=model.predict_proba(X)[:,1]
    sub_train['ans_%s'%i]=train_all
    
    test_proba=model.predict_proba(testX)[:,1]
    sub_test['ans_%s'%str(i)]=test_proba
print("mean f1 in train set:%.5f"%(ft/N))
print("mean f1 in val set:%.5f"%(fv/N))

Fold=0
[LibLinear]f1 in train set: 0.7857280016450476
f1 in test set: 0.7865267913026914
Fold=1
[LibLinear]f1 in train set: 0.785658990913077
f1 in test set: 0.786928744922744
Fold=2
[LibLinear]f1 in train set: 0.7858141915543816
f1 in test set: 0.7855366340939671
Fold=3
[LibLinear]f1 in train set: 0.7859385074252317
f1 in test set: 0.7841156610760184
Fold=4
[LibLinear]f1 in train set: 0.7858188328222621
f1 in test set: 0.7855238820585356
Fold=5
[LibLinear]f1 in train set: 0.7858597968286891
f1 in test set: 0.7851991091678076
Fold=6
[LibLinear]f1 in train set: 0.7856910652697536
f1 in test set: 0.7862614114093686
Fold=7
[LibLinear]f1 in train set: 0.7858813462204262
f1 in test set: 0.7848794069394618
Fold=8
[LibLinear]f1 in train set: 0.7857494957192842
f1 in test set: 0.786403082547859
Fold=9
[LibLinear]f1 in train set: 0.7857168773602876
f1 in test set: 0.7864028066351033
mean f1 in train set:0.78579
mean f1 in val set:0.78578


In [15]:
sub_train['label']=0
for i in range(N):
    sub_train['label']+=sub_train['ans_%s'%str(i)]
sub_train['label']=sub_train['label']/N

In [16]:
sub_test['label']=0
for i in range(N):
    sub_test['label']+=sub_test['ans_%s'%str(i)]
sub_test['label']=sub_test['label']/N

In [18]:
print(sub_train.shape,sub_test.shape)

(2049998, 11) (50000, 11)


In [20]:
sub_train['label'].to_csv('../data/ensemble/lr_train.csv',index=True)
sub_test['label'].to_csv('../data/ensemble/lr_test.csv',index=True)